In [1]:
# if the following command generates an error, you probably didn't enable 
# the cluster security option "Allow API access to all Google Cloud services"
# under Manage Security → Project Access when setting up the cluster
!gcloud dataproc clusters list --region us-central1

NAME          PLATFORM  PRIMARY_WORKER_COUNT  SECONDARY_WORKER_COUNT  STATUS   ZONE           SCHEDULED_DELETE
cluster-7aa2  GCE       2                                             RUNNING  us-central1-a


# Imports & Setup

In [2]:
!pip install -q google-cloud-storage==1.43.0
!pip install -q graphframes

In [3]:
import pyspark
import sys
from collections import Counter, OrderedDict, defaultdict
import itertools
from itertools import islice, count, groupby
import pandas as pd
import os
import re
from operator import itemgetter
import nltk
from nltk.stem.porter import *
from nltk.corpus import stopwords
from time import time
from pathlib import Path
import pickle
import pandas as pd
from google.cloud import storage
import numpy as np

import hashlib
def _hash(s):
    return hashlib.blake2b(bytes(s, encoding='utf8'), digest_size=5).hexdigest()

nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [4]:
# if nothing prints here you forgot to include the initialization script when starting the cluster
!ls -l /usr/lib/spark/jars/graph*

-rw-r--r-- 1 root root 247882 Mar  9 14:45 /usr/lib/spark/jars/graphframes-0.8.2-spark3.1-s_2.12.jar


In [5]:
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark import SparkContext, SparkConf, SparkFiles
from pyspark.sql import SQLContext
from graphframes import *

In [6]:
spark

In [7]:
# Put your bucket name below and make sure you can access it without an error
bucket_name = 'my_a3_bucket' 
full_path = f"gs://{bucket_name}/"
paths=[]

client = storage.Client()
blobs = client.list_blobs(bucket_name)
for b in blobs:
    if b.name != 'graphframes.sh' and b.name != 'init_file.sh' and b.name != "doc_len_dict.pkl" and b.name != "docid_title_dict.pkl":
        paths.append(full_path+b.name)

# Building an inverted index

Here, we read the entire corpus to an rdd, directly from Google Storage Bucket and use your code from Colab to construct an inverted index.



In [8]:
# first we try to make an inverted index on the titles.
parquetFile = spark.read.parquet(*paths)
doc_title_pairs = parquetFile.select("title", "id").rdd

We will count the number of pages to make sure we are looking at the entire corpus. The number of pages should be more than 6M

In [9]:
# Count number of wiki pages
N = parquetFile.count()
N

6348910

Let's import the inverted index module. Note that you need to use the staff-provided version called `inverted_index_gcp.py`, which contains helper functions to writing and reading the posting files similar to the Colab version, but with writing done to a Google Cloud Storage bucket.

In [10]:
# if nothing prints here you forgot to upload the file inverted_index_gcp.py to the home dir
%cd -q /home/dataproc
!ls inverted_index_gcp.py

inverted_index_gcp.py


In [11]:
# adding our python module to the cluster
sc.addFile("/home/dataproc/inverted_index_gcp.py")
sys.path.insert(0,SparkFiles.getRootDirectory())

In [12]:
from inverted_index_gcp import InvertedIndex

In [13]:
english_stopwords = frozenset(stopwords.words('english'))
# corpus_stopwords = ["category", "references", "also", "external", "links", 
#                     "may", "first", "see", "history", "people", "one", "two", 
#                     "part", "thumb", "including", "second", "following", 
#                     "many", "however", "would", "became"]
# all_stopwords = english_stopwords.union(corpus_stopwords)
all_stopwords = english_stopwords
RE_WORD = re.compile(r"""[\#\@\w](['\-]?\w){2,24}""", re.UNICODE)

In [14]:
NUM_BUCKETS = 124
def token2bucket_id(token):
    return int(_hash(token),16) % NUM_BUCKETS

def word_count(text, id):
    ''' Count the frequency of each word in `text` (tf) that is not included in
    `all_stopwords` and return entries that will go into our posting lists.
    Parameters:
    -----------
    text: str
    Text of one document
    id: int
    Document id
    Returns:
    --------
    List of tuples
    A list of (token, (doc_id, tf)) pairs
    for example: [("Anarchism", (12, 5)), ...]
    '''
    
    # Base tokenizer
    tokens = [token.group() for token in RE_WORD.finditer(text.lower())]
    
    # Filter out tokens that are in the stopwords set
    filtered_tokens = [token for token in tokens if token not in all_stopwords]


    # Count the frequency of each remaining token
    token_counts = Counter(filtered_tokens)

    # Format the result as a list of (token, (doc_id, tf)) tuples
    posting_list = [(token, (id, int(tf))) for token, tf in token_counts.items()]

    return posting_list

In [15]:
word_counts = doc_title_pairs.flatMap(lambda x: word_count(x[0], x[1]))

In [16]:
def reduce_word_counts(unsorted_pl):
    ''' Returns a sorted posting list by wiki_id.
    Parameters:
    -----------
    unsorted_pl: list of tuples
      A list of (wiki_id, tf) tuples
    Returns:
    --------
    list of tuples
      A sorted posting list.
    '''
    return sorted(unsorted_pl, key=lambda pair: pair[0])

In [17]:
postings = word_counts.groupByKey().mapValues(reduce_word_counts)

In [18]:
# Filter rare words
# min_term_fr = 50
# postings_filtered = postings.filter(lambda x: len(x[1])>min_term_fr)
postings_filtered = postings

In [19]:
def calculate_df(postings):
    ''' Takes a posting list RDD and calculate the df for each token.
    Parameters:
    -----------
    postings: RDD
      An RDD where each element is a (token, posting_list) pair.
    Returns:
    --------
    RDD
      An RDD where each element is a (token, df) pair.
    '''
    return postings.map(lambda x: (x[0], len([doc_id for doc_id, _ in x[1]])))

In [20]:
w2df_rdd = calculate_df(postings_filtered)

In [21]:
def calculate_document_lengths_from_postings(postings_filtered):
    """
    Calculate the approximate length of each document based on term frequencies
    in the postings_filtered RDD.

    Parameters:
    postings_filtered: RDD of (term, [(doc_id, tf), ...]) pairs.

    Returns:
    RDD of (doc_id, doc_len) pairs, where doc_len is the sum of term frequencies for each document.
    """
    # Flatten the postings to (doc_id, tf) pairs
    doc_tf_pairs = postings_filtered.flatMap(lambda term_postings: [(doc_id, tf) for doc_id, tf in term_postings[1]])
    
    # Sum the term frequencies for each document
    doc_lengths = doc_tf_pairs.reduceByKey(lambda tf1, tf2: tf1 + tf2)
    
    return doc_lengths

doc_lengths = calculate_document_lengths_from_postings(postings_filtered)

In [22]:
# Calculating avarage document length
# total_doc_len = doc_lengths.map(lambda x: x[1]).reduce(lambda a, b: a + b)
# avg_doc_len = total_doc_len / N

# # Calculate document length
# doc_len_dict = doc_lengths.collectAsMap()

In [23]:
# def bm25_doc_term_score(postings_filtered, N, avg_doc_len, doc_len_dict, b=0.2, k1=1):
#     # Define the BM25 score calculation for a term in a single document
#     def calculate_bm25(tf, df, doc_len, avg_doc_len):
#         idf = np.log(N+1) / df
#         B = 1 - b + b * (doc_len / avg_doc_len)
#         return int((idf * ((k1 + 1) * tf) / (B * k1 + tf)) * 2**20)
    
#         # GPT says this is the real calculation of the score
#         #  idf = np.log((N - df + 0.5) / (df + 0.5))  # Using the standard BM25 idf formula
#         #  B = 1 - b + b * (doc_len / avg_doc_len)
#         #  return idf * ((k1 + 1) * tf) / ((k1 * B) + tf)

#     # Define a function to apply BM25 to all terms in a document
#     def apply_bm25_to_doc_terms(term_postings):
#         term, postings_list = term_postings
#         df = len(postings_list)
#         # Process each document
#         updated_postings = [
#             (doc_id, calculate_bm25(tf, df, doc_len_dict.get(doc_id, 0), avg_doc_len))
#             for doc_id, tf in postings_list]  # Ensure doc_lengths are accessed correctly
#         return term, updated_postings

#     # Apply the BM25 calculation to each term-document pair in the RDD
#     bm25_scores = postings_filtered.map(apply_bm25_to_doc_terms)
#     return bm25_scores


In [24]:
# postings_filtered RDD
# final_postings_filtered = bm25_doc_term_score(postings_filtered, N, avg_doc_len, doc_len_dict)
final_postings_filtered = postings_filtered
# postings_filtered Dict
# final_postings_filtered_as_dict = final_postings_filtered.collectAsMap()

In [25]:
def partition_postings_and_write(postings):
    ''' A function that partitions the posting lists into buckets, writes out
    all posting lists in a bucket to disk, and returns the posting locations for
    each bucket. Partitioning should be done through the use of `token2bucket`
    above. Writing to disk should use the function  `write_a_posting_list`, a
    static method implemented in inverted_index_colab.py under the InvertedIndex
    class.
    Parameters:
    -----------
    postings: RDD
      An RDD where each item is a (w, posting_list) pair.
    Returns:
    --------
    RDD
      An RDD where each item is a posting locations dictionary for a bucket. The
      posting locations maintain a list for each word of file locations and
      offsets its posting list was written to. See `write_a_posting_list` for
      more details.
    '''

    # map postings to buckets
    bucketed_postings = postings.map(lambda x: (token2bucket_id(x[0]), x))

    # group postings by bucket
    grouped_postings = bucketed_postings.groupByKey().mapValues(list)

    base_dir = "./title_posting_locs"

    # write postings for each bucket to disk and collect location info
    def map_write_bucket_postings(bucket_postings):
        bucket_id, postings_list = bucket_postings
        # Call write_a_posting_list with the correctly formatted argument
        posting_locations = InvertedIndex.write_a_posting_list((bucket_id, postings_list), base_dir, bucket_name)
        return posting_locations

    # apply write_bucket_postings to each group of postings
    posting_locations_rdd = grouped_postings.map(map_write_bucket_postings)

    return posting_locations_rdd

In [26]:
t_start = time()
_ = partition_postings_and_write(final_postings_filtered).collect()
index_const_time = time() - t_start

In [27]:
# time the index creation time
# t_start = time()
# word counts map
# word_counts = doc_text_pairs.flatMap(lambda x: word_count(x[0], x[1]))
# postings = word_counts.groupByKey().mapValues(reduce_word_counts)
# filtering postings and calculate df
# postings_filtered = postings.filter(lambda x: len(x[1])>50)
# w2df = calculate_df(postings_filtered)
# w2df_dict = w2df.collectAsMap()
# partition posting lists and write out
# _ = partition_postings_and_write(postings_filtered).collect()
# index_const_time = time() - t_start

In [28]:
# collect all posting lists locations into one super-set
super_posting_locs = defaultdict(list)
for blob in client.list_blobs(bucket_name, prefix='title_posting_locs'):
    if not blob.name.endswith("pickle"):
        continue
    with blob.open("rb") as f:
        posting_locs = pickle.load(f)
        for k, v in posting_locs.items():
            super_posting_locs[k].extend(v)

Putting it all together

In [29]:
# Create inverted index instance
inverted = InvertedIndex()
# Adding the posting locations dictionary to the inverted index
inverted.posting_locs = super_posting_locs
# Add the token - df dictionary to the inverted index
w2df_dict = w2df_rdd.collectAsMap()
inverted.df = w2df_dict
# inverted.N = N
# Adding the doc_id:title dictionary
# doc_title_pairs_dict = doc_title_pairs.map(lambda row: (row.id, row.title)).collectAsMap()
# inverted.doc_id_title_dict = doc_title_pairs_dict
# write the global stats out
inverted.write_index('.', 'title_index')
# upload to gs
index_src = "title_index.pkl"
index_dst = f'gs://{bucket_name}/title_posting_gcp/{index_src}'
!gsutil cp $index_src $index_dst

Copying file://title_index.pkl [Content-Type=application/octet-stream]...
- [1 files][ 67.6 MiB/ 67.6 MiB]                                                
Operation completed over 1 objects/67.6 MiB.                                     


In [30]:
!gsutil ls -lh $index_dst

 67.61 MiB  2024-03-09T18:31:13Z  gs://my_a3_bucket/title_posting_gcp/title_index.pkl
TOTAL: 1 objects, 70890671 bytes (67.61 MiB)


### Loading a pickle for bucket and reading posting list

In [31]:
from inverted_index_gcp import *
index_src = "title_index.pkl"
index_path = f"title_posting_gcp/{index_src}"
postings_path = ""
bucket = get_bucket(bucket_name)
blob = bucket.blob(index_path)
contents = blob.download_as_bytes()
inverted = pickle.loads(contents)

In [32]:
inverted.read_a_posting_list(base_dir=postings_path,w='game',bucket_name=bucket_name)

[(785, 1),
 (1136, 1),
 (1183, 1),
 (3401, 1),
 (5360, 1),
 (5362, 1),
 (5363, 1),
 (5722, 1),
 (5743, 1),
 (6259, 1),
 (7431, 1),
 (8162, 1),
 (8484, 1),
 (8521, 1),
 (10503, 1),
 (11157, 1),
 (11924, 1),
 (11979, 1),
 (12372, 1),
 (12430, 1),
 (12919, 1),
 (13097, 1),
 (13135, 1),
 (13611, 1),
 (13829, 1),
 (14199, 1),
 (14462, 1),
 (16529, 1),
 (16789, 1),
 (17560, 1),
 (18666, 1),
 (19692, 1),
 (19871, 1),
 (20434, 1),
 (20844, 1),
 (23472, 1),
 (23654, 1),
 (25266, 1),
 (25475, 1),
 (26459, 1),
 (26570, 1),
 (26809, 1),
 (26914, 1),
 (27541, 1),
 (28355, 1),
 (28760, 1),
 (28918, 1),
 (29027, 1),
 (29538, 1),
 (29947, 1),
 (29950, 1),
 (30245, 1),
 (30492, 1),
 (30581, 1),
 (30689, 1),
 (31574, 1),
 (32193, 1),
 (32393, 1),
 (32394, 1),
 (32398, 1),
 (32399, 1),
 (32629, 1),
 (32722, 1),
 (34417, 1),
 (34678, 1),
 (36703, 1),
 (36867, 1),
 (37032, 1),
 (37035, 1),
 (37068, 1),
 (38373, 1),
 (39292, 1),
 (42661, 1),
 (43277, 1),
 (49093, 1),
 (49626, 1),
 (49827, 1),
 (51959, 1),
 

# PageRank

**YOUR TASK (10 POINTS):** Compute PageRank for the entire English Wikipedia. Use your implementation for `generate_graph` function from Colab below.

In [ ]:
def generate_graph(pages):
  '''
  Compute the directed graph generated by wiki links.
  Parameters:
  -----------
    pages: RDD
      An RDD where each row consists of one wikipedia articles with 'id' and
      'anchor_text'.
  Returns:
  --------
    edges: RDD
      An RDD where each row represents an edge in the directed graph created by
      the wikipedia links. The first entry should the source page id and the
      second entry is the destination page id. No duplicates should be present.
    vertices: RDD
      An RDD where each row represents a vetrix (node) in the directed graph
      created by the wikipedia links. No duplicates should be present.
  '''

  edges = pages.flatMap(lambda page: [(page.id, anchor.id) for anchor in page.anchor_text]).distinct()

  vertices_from_edges_source = edges.map(lambda edge: (edge[0],))
  vertices_from_edges_destination = edges.map(lambda edge: (edge[1],))

  vertices = pages.map(lambda page: (page.id,)).union(vertices_from_edges_source).union(vertices_from_edges_destination).distinct()

  return edges, vertices

In [ ]:
t_start = time()
pages_links = parquetFile.select ("id","anchor_text").rdd
# construct the graph 
edges, vertices = generate_graph(pages_links)
# compute PageRank
edgesDF = edges.toDF(['src', 'dst']).repartition(124, 'src')
verticesDF = vertices.toDF(['id']).repartition(124, 'id')
g = GraphFrame(verticesDF, edgesDF)
pr_results = g.pageRank(resetProbability=0.15, maxIter=6)
pr = pr_results.vertices.select("id", "pagerank")
pr = pr.sort(col('pagerank').desc())
pr.repartition(1).write.csv(f'gs://{bucket_name}/pr', compression="gzip")
pr_time = time() - t_start
pr.show()

In [ ]:
# test that PageRank computaion took less than 1 hour
assert pr_time < 60*60*2

# Reporting

**YOUR TASK (5 points):** execute and complete the following lines to complete 
the reporting requirements for assignment #3. 

In [ ]:
# size of input data
!gsutil du -sh "gs://wikidata_preprocessed/"

In [ ]:
# size of index data
index_dst = f'gs://{bucket_name}/postings_gcp/'
!gsutil du -sh "$index_dst"

In [ ]:
# How many USD credits did you use in GCP during the course of this assignment?
cost = 4.05
print(f'I used {cost} USD credit during the course of this assignment')

**Bonus (10 points)** if you implement PageRank in pure PySpark, i.e. without using the GraphFrames package, AND manage to complete 10 iterations of your algorithm on the entire English Wikipedia in less than an hour. 


In [ ]:
#If you have decided to do the bonus task - please copy the code here 

bonus_flag = False # Turn flag on (True) if you have implemented this part

t_start = time()

# PLACE YOUR CODE HERE

pr_time_Bonus = time() - t_start


In [ ]:
# Note:test that PageRank computaion took less than 1 hour
assert pr_time_Bonus < 60*60 and bonus_flag